In [1]:
# HW4 - Question 2

from LOB_GAN_testing_lxynew import analyze_microstructure

stock_list = ["0050", "0056", "2330"]

results_q2 = {}

for stock in stock_list:
    print(f"========== {stock} (raw model) ==========")
    stats_raw, ks_raw = analyze_microstructure(
        stock_code=stock,
        version="raw",       # 只用原始模型就够完成 Q2，如想比较 adjusted 可以再跑一遍
        threshold=0.5, # 10% quantile
        batch_size=1,
        seed=307,
        save_kde_fig=True,   # 顺便存一张 trade_return KDE 图，可选
    )
    results_q2[stock] = {
        "stats": stats_raw,
        "ks": ks_raw,
    }
    
print(stats_raw.head())
print(ks_raw.head())

========== 0050 (raw model) ==========
Raw data loading and processing 0050
Data 1 for 0050 loaded.
Data 2 for 0050 loaded.
Data 3 for 0050 loaded.
Minutely data generated.
Loaded models from /Users/wenlan/python-github/Anomaly-Detection/Results/0050_generator1.pth and /Users/wenlan/python-github/Anomaly-Detection/Results/0050_discriminator1.pth
Total days: 55, abnormal days: 0 (0.00%)
Saved KDE figure to: /Users/wenlan/python-github/Anomaly-Detection/Results/0050_trade_return_KDE_raw.png
========== 0056 (raw model) ==========
Raw data loading and processing 0056
Data 1 for 0056 loaded.
Data 2 for 0056 loaded.
Data 3 for 0056 loaded.
Minutely data generated.
Loaded models from /Users/wenlan/python-github/Anomaly-Detection/Results/0056_generator1.pth and /Users/wenlan/python-github/Anomaly-Detection/Results/0056_discriminator1.pth
Total days: 55, abnormal days: 55 (100.00%)
Saved KDE figure to: /Users/wenlan/python-github/Anomaly-Detection/Results/0056_trade_return_KDE_raw.png
=========